In [1]:
!pip install BeatNet
!pip install git+https://github.com/CPJKU/madmom.git
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 22.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of beatnet to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cuf

In [2]:
# Uninstall the current version of NumPy
!pip uninstall numpy -y
# Install the specific version of NumPy
!pip install numpy==1.26.4

Found existing installation: numpy 1.25.2
Uninstalling numpy-1.25.2:
  Successfully uninstalled numpy-1.25.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 25.8 MB/s eta 0:00:00


In [1]:
import numpy
print(numpy.__version__)

1.26.4


In [ ]:
from BeatNet.BeatNet import BeatNet

estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)

Output = estimator.process("/content/audios/SMC_001.wav")

In [ ]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 22.5 MB/s eta 0:00:00


In [188]:
import torch
import pytorch_lightning as pl

class BasicModel(pl.LightningModule):
    def __init__(
            self,
            model,
            milestones: int = 5,
            batch_size: int = 32,
            lr: float = 1e-3,
            lr_scheduler_gamma: float = 1e-1,
            lr_patience = 3,
            min_lr=0,
            name='BasicModel',
            **kwargs,
    ) -> None:
        super().__init__()
        self.name = name
        self.lr = lr
        self.lr_patience = lr_patience
        self.batch_size = batch_size
        self.min_lr = min_lr
        self.lr_scheduler_gamma = lr_scheduler_gamma
        self.milestones = milestones
        self.model = model
        self.accuracy = self.setup_accuracy()
        self.test_y = []
        self.test_y_hat = []


    def setup_accuracy(self,):
        return Accuracy(task="binary")

    def forward(self, x):
        y_hat = self.model(x)
        return y_hat

    def loss(self, y_hat, y):
        self.loss_func = nn.BCEWithLogitsLoss()
        return self.loss_func(y_hat, y)

    def setup_optimizer(self, optimizer):
        self.optimizer = optimizer

    def setup_lr_scheduler(self,):
        return torch.optim.lr_scheduler.ReduceLROnPlateau

    def configure_optimizers(self):
        lr_scheduler_type = self.setup_lr_scheduler()
        self.lr_scheduler = lr_scheduler_type(self.optimizer, patience=self.lr_patience, min_lr=self.min_lr)
        return {"optimizer": self.optimizer, "lr_scheduler": self.lr_scheduler, "monitor": "val_loss"}

    def _log_all(self, stage, loss, y_hat, y):
        dict_for_logging = {f'{stage}_acc': self.accuracy(y_hat, y)}
        dict_for_logging[f'{stage}_loss'] = loss
        dict_for_logging['batch_size'] = self.batch_size

        # if on_epoch=False, EarlyStopping callback may throw an error
        self.log_dict(dict_for_logging, on_step=False, on_epoch=True, prog_bar=True)

    def _step(self, stage, batch):
        x, y, _ = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        self._log_all(stage, loss, y_hat, y)

        if stage=='test':
            self.test_y.append(y)
            self.test_y_hat.append(y_hat)

        return loss

    def training_step(self, batch, batch_idx):
        return self._step('train', batch)

    def validation_step(self, batch, batch_idx):
        return self._step('val', batch)

    def test_step(self, batch, batch_idx):
        loss = self._step('test', batch)

In [151]:
from pytorch_lightning.callbacks import BaseFinetuning

class FineTuning(BaseFinetuning):
    # freezes feature_extractor
    def __init__(self, unfreeze_at_epoch_n_layers_dict):
        super().__init__()
        self.unfreeze_at_epoch_n_layers_dict = unfreeze_at_epoch_n_layers_dict

    def on_fit_start(self, trainer, pl_module):
        super().on_fit_start(trainer, pl_module)
        total_layers = len(list(pl_module.modules()))
        assert all([n_layers <= total_layers for n_layers in self.unfreeze_at_epoch_n_layers_dict.values()]), f"Provided too many layers for unfreezing"

    def freeze_before_training(self, pl_module):
        self.freeze(pl_module)
        BaseFinetuning.make_trainable(pl_module.model.linear)

    def finetune_function(self, pl_module, current_epoch, optimizer):

        if current_epoch in self.unfreeze_at_epoch_n_layers_dict.keys():
            i = -self.unfreeze_at_epoch_n_layers_dict[current_epoch]
            self.unfreeze_and_add_param_group(
                modules=list(pl_module.modules())[i:],
                optimizer=optimizer,
                train_bn=True,
            )
            pl_module.lr_scheduler.min_lrs = [pl_module.min_lr] * len(optimizer.param_groups)

In [192]:
from ast import Return
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
from datetime import datetime
import os
import utils
import torch
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter

def plot_predictions(paths, y_hat, labels, save_dir, class_names):
    num_images = len(paths)
    for i, path in enumerate(paths):
        img = Image.open(path)

        # Create a new figure for each image
        fig = plt.figure()

        # Plot the image
        plt.imshow(img)
        # Don’t call .item() anywhere in your code. Use .detach() instead to remove the connected graph calls.
        pred = torch.argmax(y_hat[i].detach())
        true = labels[i].detach()
        plt.title(f'Pred: {class_names[pred]}, True: {class_names[true]}')
        plt.axis('off')

        # Save the figure
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f'image_{i}.png')
        fig.savefig(save_path)
        plt.close(fig)

class Experiment():
    def __init__(self, name, ds_dir='', model_path='', train_split=0.8, val_split=0.1,
                 load_from_hps_flag=False, save_exp_hps_flag=True, save_confusion_matrix_flag=True,
                 num_workers=0, lr_patience=2, lr=1e-3, min_lr=0., n_epochs_max = -1, unfreeze_at_epoch_n_layers_dict = {5:5, 10:18, 15:50, 20:80},
                 early_stopping_patience=3, logs_dir='lightning_logs', checkpoints_dir='lightning_checkpoints',
                 batch_size=8, seed=10, weight_decay=0.01):

        '''
        If you want to load the experiment, provide the full name of the experiment you want to load.
        Please note that it won't load the trained weights! Only the experiment
        '''
        self.train_split = train_split
        self.val_split = val_split
        self.experiment_settings_set_up = False
        self.load_from_hps_flag = load_from_hps_flag
        if load_from_hps_flag:
            self.name = name
            self.work_dir = os.path.join('experiment_runs', f'{self.name}_work_dir')
            self.load_from_hps()
            self.setup_experiment_settings()
            Return
        self.ds_dir = ds_dir
        self.model_path = model_path
        self.num_workers = num_workers
        self.lr_patience = lr_patience
        self.lr = lr
        self.min_lr = min_lr
        self.n_epochs_max = n_epochs_max
        self.unfreeze_at_epoch_n_layers_dict = unfreeze_at_epoch_n_layers_dict
        self.early_stopping_patience = early_stopping_patience
        self.logs_dir = logs_dir
        self.checkpoints_dir = checkpoints_dir
        self.save_confusion_matrix_flag = save_confusion_matrix_flag
        self.save_exp_hps_flag = save_exp_hps_flag
        self.batch_size = batch_size
        self.name = f'{name}_{datetime.now().strftime("%Hh%Mm")}'
        self.work_dir = os.path.join('experiment_runs', f'{self.name}_work_dir')
        self.seed = seed
        self.weight_decay = weight_decay
        os.makedirs(self.work_dir, exist_ok=True)

    def get_data_module(self,):
        return BeatsDataModule(ds_dir=ds_dir, labels_dir=labels_dir, train_split=self.train_split, val_split=self.val_split,
                               batch_size=self.batch_size)

    def get_model(self,):
        model = BasicModel(BDA(272, 150, 2, 'cpu'))
        return model

    def get_optimizer(self, model):
        return torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()), lr=self.lr, weight_decay=self.weight_decay)

    def get_callbacks(self,):
        fine_tuning = FineTuning(unfreeze_at_epoch_n_layers_dict={})
        checkpoint = ModelCheckpoint(dirpath=self.checkpoints_dir, filename=self.name + '_{epoch:02d}-{val_loss:.2f}',
                                     save_top_k=1, monitor='val_loss', mode='min')
        early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=self.early_stopping_patience)
        return [fine_tuning, checkpoint, early_stopping]

    def get_loggers(self,):
        tensorboard = TensorBoardLogger(name=self.name, save_dir=self.logs_dir)
        csv = CSVLogger(name=self.name, save_dir=self.logs_dir)
        return [tensorboard, csv]

    def load_trainer(self, **kwargs):
        return pl.Trainer(max_epochs=self.n_epochs_max,callbacks=self.get_callbacks(),logger=self.get_loggers(),)

    def setup_experiment_settings(self,):
        if not self.load_from_hps_flag:
            self.data_module = self.get_data_module()
        else:
            self.data_module = self.get_data_module_from_pickles()
        pl.seed_everything(self.seed, workers=True)
        self.model = self.get_model()
        self.optimizer = self.get_optimizer(self.model)
        self.model.optimizer = self.optimizer
        self.trainer = self.load_trainer()
        self.experiment_settings_set_up = True

    def fit(self,):
        if not self.experiment_settings_set_up:
            self.setup_experiment_settings()
        self.trainer.fit(self.model, self.data_module)

    def test(self,):
        if not self.experiment_settings_set_up:
            self.setup_experiment_settings()
        self.trainer.test(self.model, self.data_module)
        #if self.save_confusion_matrix_flag:
            #self.save_confusion_matrix()

    def train(self,):
        self.fit()
        self.test()
        if self.save_exp_hps_flag:
            self.save_experiment_hps()

    def get_train_paths_list(self,):
        return self.data_module.train_samples

    def get_val_paths_list(self):
        return self.data_module.val_samples

    def get_test_paths_list(self):
        return self.data_module.test_samples

    def pickle_splits(self, ):
        train_file = os.path.join(self.work_dir, 'train_file_paths.pkl')
        utils.pickle_it_as(self.get_train_paths_list(), train_file)

        val_file = os.path.join(self.work_dir, 'val_file_paths.pkl')
        utils.pickle_it_as(self.get_val_paths_list(), val_file)

        test_file = os.path.join(self.work_dir, 'test_file_paths.pkl')
        utils.pickle_it_as(self.get_test_paths_list(), test_file)

    def get_pickled_splits(self):
        train_file = os.path.join(self.work_dir, 'train_file_paths.pkl')
        train_paths_list = utils.get_pickle(train_file)

        val_file = os.path.join(self.work_dir, 'val_file_paths.pkl')
        val_paths_list = utils.get_pickle(val_file)

        test_file = os.path.join(self.work_dir, 'test_file_paths.pkl')
        test_paths_list = utils.get_pickle(test_file)

        return train_paths_list, val_paths_list, test_paths_list

    def get_data_module_from_pickles(self):
        self.data_module = self.get_data_module()
        train_paths_list, val_paths_list, test_paths_list = self.get_pickled_splits()
        self.data_module.train_samples = train_paths_list
        self.data_module.val_samples = val_paths_list
        self.data_module.test_samples = test_paths_list

    def save_confusion_matrix(self):
        cm_name = os.path.join(self.work_dir, f'confusion_matrix_{self.name}.png')
        cm_name = os.path.join(self.work_dir, f'confusion_matrix_{self.name}.pdf')
        self.model.cm_fig.savefig(cm_name, dpi=self.model.cm_fig.dpi, bbox_inches='tight')

    def forward_cam_batch(self, batch):
        x, labels, paths = batch
        return self.model(x), labels, paths

    def save_examples_from_batch_of_split(self, split_name):
        if split_name == 'train':
            save_dir = os.path.join(self.work_dir, 'train_prediction_examples')
        if split_name == 'val':
            save_dir = os.path.join(self.work_dir, 'val_prediction_examples')
        if split_name == 'test':
            save_dir = os.path.join(self.work_dir, 'test_prediction_examples')
        else:
            save_dir = os.path.join(self.work_dir, 'prediction_examples')

        for batch in self.data_module.get_cam_dataloader(split_name):
            y_hat, labels, paths = self.forward_cam_batch(batch)
            plot_predictions(paths, y_hat, labels, save_dir, self.data_module.class_names)
            break

    def save_experiment_hps(self,):
        attributes = self.__dict__.copy()
        attributes.pop('experiment_settings_set_up', None)
        attributes.pop('data_module', None)
        attributes.pop('model', None)
        attributes.pop('trainer', None)

        hps_file = os.path.join(self.work_dir, 'experiment_hps.pkl')
        utils.pickle_it_as(attributes, hps_file)

        self.pickle_splits()

    def load_from_hps(self):
        hps_file = os.path.join(self.work_dir, 'experiment_hps.pkl')
        attributes = utils.get_pickle(hps_file)
        self.__dict__.update(attributes)

    def load_fine_tuned_weights_from_checkpoint(self, checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        self.model.load_state_dict(checkpoint['state_dict'])

    def load_checkpoint(self, checkpoint_path):
        '''self.model = self.model.__class__.load_from_checkpoint(checkpoint_path=checkpoint_path)
        self.trainer = pl.Trainer()'''
        pass

In [ ]:
from torchmetrics import Accuracy

In [19]:
import os
import pickle
import cv2

img_exts = ['.jpg', '.jpeg', '.gif', '.bmp', '.png']


def get_files_with_ext_in_dir(dir_path, exts=None, recursive=True, exceptions=['.ipynb_checkpoints', '@eaDir', '__pycache__']):
    found_files = []
    files = os.listdir(dir_path)

    for file in files:
        file_path = os.path.join(dir_path, file)
        if any((exception in file_path for exception in exceptions)):
            continue

        if recursive and os.path.isdir(file_path):
            found_files.extend(get_files_with_ext_in_dir(file_path, exts, recursive))
        elif exts is None or any(file.lower().endswith(ext) for ext in exts):
            found_files.append(file_path)

    return found_files

In [20]:
labels_dir = 'labels'
ds_dir = 'audios'

In [21]:
import torchaudio
import torch

def ensure_mono(wav_tensor, from_bits):
    if len(wav_tensor.squeeze().shape) > 1:
        wav_tensor = torch.mean(wav_tensor, dim=0)
        wav_tensor = wav_tensor.to(dtype=bit_types[from_bits])
    return wav_tensor.squeeze()

def ensure_sample_rate(original_sample_rate, waveform, desired_sample_rate=22050):
    if original_sample_rate != desired_sample_rate:
        resampler = torchaudio.transforms.Resample(original_sample_rate, desired_sample_rate, dtype=waveform.dtype)
        waveform = resampler(waveform)
    return desired_sample_rate, waveform

def load_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path, normalize=True)

    metadata = torchaudio.info(file_path)
    from_bits = metadata.bits_per_sample
    waveform = ensure_mono(waveform, from_bits)

    sample_rate, waveform = ensure_sample_rate(sample_rate, waveform, desired_sample_rate=22050)
    return sample_rate, waveform.numpy()

In [28]:
def get_beats_stamps_from_path(label_path):
    beat_time_stamps = []
    with open(label_path) as f:
        lines = f.readlines()
    for line in lines:
        beat_time_stamps.append([float(line), 1])
    return beat_time_stamps

In [4]:
from BeatNet.log_spect import LOG_SPECT

def convert_to_spectrogram(audio, sample_rate):
  hop_length = int(20 * 0.001 * sample_rate)
  win_length = int(64 * 0.001 * sample_rate)
  to_spect = LOG_SPECT(sample_rate=sample_rate, win_length=win_length, hop_size=hop_length, n_bands=[24], mode = 'offline')
  return to_spect.process_audio(audio).T, hop_length

In [5]:
def beat_stamps_to_labels(audio, hop_length, beat_time_stamps, sample_rate):
  total_time_on_spec = int(len(audio) / hop_length)
  labels = torch.zeros((1, total_time_on_spec))
  for beat_time_stamp in beat_time_stamps:
    time_moment, cl = beat_time_stamp
    time_moment_on_spec = int(time_moment * sample_rate / hop_length)
    labels[0, time_moment_on_spec] = 1
  return labels

In [113]:
import os
from torch.utils.data import Dataset

class AudioBeatsDataset(Dataset):
    def __init__(self, audio_paths, labels_dir, transform=None):
        self.audio_paths = audio_paths
        self.labels_dir = labels_dir
        self.transform = transform

    def __len__(self):
        return len(self.audio_paths)

    def __getitem__(self, idx):
        audio_path = self.audio_paths[idx]
        sample_rate, audio_data = load_audio(audio_path)
        if self.transform:
            audio_data = self.transform(audio_data)

        path_to_find = os.path.join(self.labels_dir, os.path.splitext(os.path.basename(audio_path))[0])
        matching_files = []
        labels_path = None
        for root, dirs, files in os.walk(self.labels_dir):
            for file in files:
                file = os.path.join(root, file)
                if file.startswith(path_to_find):
                    labels_path = file

        beat_time_stamps = get_beats_stamps_from_path(labels_path)

        spectrogram, hop_length = convert_to_spectrogram(audio_data, sample_rate)
        labels = beat_stamps_to_labels(audio_data, hop_length, beat_time_stamps, sample_rate)
        feats = torch.from_numpy(spectrogram)
        feats = feats.unsqueeze(0).float()

        labels = labels.to('cpu').squeeze(dim=0).float()

        return feats, labels, audio_path

In [12]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 17.1 MB/s eta 0:00:00


In [127]:
import ds_utils
import pytorch_lightning as pl
from torch.utils.data import DataLoader

class BeatsDataModule(pl.LightningDataModule):
    '''
    Expected basic structure with no subdirs.
    '''
    def __init__(self, ds_dir, labels_dir, train_split, val_split, batch_size, num_workers=0, seed=None):
        super().__init__()
        self.ds_dir = ds_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.seed = seed
        self.labels_dir = labels_dir
        self.train_samples, self.val_samples, self.test_samples = self._process_and_split(train_split, val_split)

    def _process_and_split(self, train_split, val_split):
        # if you want specific exts, select from utils
        return ds_utils.get_split_samples_stratified(self.ds_dir, train_split, val_split, exts=['.wav'], seed=self.seed)

    def _get_ds(self, samples):
        return AudioBeatsDataset(samples, self.labels_dir)#, transform=MelSpectrogramTransform())

    def _get_dl(self, ds, shuffle):
        return DataLoader(ds, batch_size=self.batch_size, collate_fn=collate_with_tensor_preprocessing, shuffle=shuffle, num_workers=self.num_workers)

    def train_dataloader(self):
        ds = self._get_ds(self.train_samples)
        return self._get_dl(ds, True)

    def val_dataloader(self):
        ds = self._get_ds(self.val_samples)
        return self._get_dl(ds, False)

    def test_dataloader(self):
        ds = self._get_ds(self.test_samples)
        return self._get_dl(ds, False)

    def get_train_paths_list(self):
        return self.train_samples

    def get_val_paths_list(self):
        return self.val_samples

    def get_test_paths_list(self):
        return self.test_samples

def collate_with_tensor_preprocessing(batch):
    """
   batch: is a list of tuples with (wavs, masks, label, length)
    """
    wav_tensors, labels, file_names = zip(*batch)

    # Convert lists to tensors
    labels = torch.stack(labels)
    #labels = torch.tensor(labels, dtype=torch.long, device='cuda')

    # Pad sequences
    wav_batch = torch.nn.utils.rnn.pad_sequence(wav_tensors, batch_first=True, padding_value=0.0)
    wav_batch = torch.nn.utils.rnn.pad_sequence(wav_tensors, batch_first=True, padding_value=0)

    wav_batch.requires_grad = True
    labels.requires_grad = True

    return wav_batch, labels, list(file_names)

In [186]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np


class BDA(nn.Module):  #beat_downbeat_activation
    def __init__(self, dim_in, num_cells, num_layers, device):
        super(BDA, self).__init__()

        self.dim_in = dim_in
        self.dim_hd = num_cells
        self.num_layers = num_layers
        self.device = device
        self.conv_out = 150
        self.kernelsize = 10
        self.conv1 = nn.Conv1d(1, 2, self.kernelsize)
        self.linear0 = nn.Linear(2*int((self.dim_in-self.kernelsize+1)/2), self.conv_out)     #divide to 2 is for max pooling filter
        self.lstm = nn.LSTM(input_size=self.conv_out,  # self.dim_in
                            hidden_size=self.dim_hd,
                            num_layers=self.num_layers,
                            batch_first=True,
                            bidirectional=False,
                            )
        # for binary
        out_features = 1
        self.linear = nn.Linear(in_features=self.dim_hd,
                                out_features=1)

        self.softmax = nn.Softmax(dim=0)
        # Initialize the hidden state and cell state

        self.to(device)

    def forward(self, data):
        x = data
        x = torch.reshape(x, (-1, self.dim_in))
        x = x.unsqueeze(0).transpose(0, 1)
        x = F.max_pool1d(F.relu(self.conv1(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = self.linear0(x)
        x = torch.reshape(x, (np.shape(data)[0], np.shape(data)[2], self.conv_out))
        x, (self.hidden, self.cell) = self.lstm(x, (torch.zeros(2, np.shape(data)[0], self.dim_hd).to(device), torch.zeros(2, np.shape(data)[0], self.dim_hd).to(device)))
        # x = self.lstm(x)[0]
        out = self.linear(x)
        out = out.transpose(1, 2)
        out = out.to('cpu').argmax(dim=1).float()
        out.requires_grad = True
        return out

    def final_pred(self, input):
        return self.softmax(input)

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [80]:
dm = BeatsDataModule(ds_dir, labels_dir, 0.8, 0.1, 4)

In [193]:
exp = Experiment('attempt', ds_dir=ds_dir, n_epochs_max=-1)

In [194]:
exp.train()

INFO:lightning_fabric.utilities.seed:Seed set to 10
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /content/lightning_checkpoints exists and is not empty.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type           | Params
--------------------------------------------
0 | model    | BDA            | 402 K 
1 | accuracy | BinaryAccuracy | 0     
--------------------------------------------
151       Trainable params
401 K     Non-trainable params
402 K     Total params
1.608     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (22) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        batch_size         │           32.0            │
│         test_acc          │    0.9768260717391968     │
│         test_loss         │    0.6931474208831787     │
└───────────────────────────┴───────────────────────────┘

In [185]:
exp = Experiment('attempt', ds_dir=ds_dir, n_epochs_max=-1)
exp.train()

INFO:lightning_fabric.utilities.seed:Seed set to 10
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /content/lightning_checkpoints exists and is not empty.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type           | Params
--------------------------------------------
0 | model    | BDA            | 402 K 
1 | accuracy | BinaryAccuracy | 0     
--------------------------------------------
302       Trainable params
401 K     Non-trainable params
402 K     Total params
1.609     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (22) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        batch_size         │           32.0            │
│         test_acc          │    0.02456521801650524    │
│         test_loss         │     366.5239562988281     │
└───────────────────────────┴───────────────────────────┘